In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [4]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [5]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

small_ratingsValid = ratingsValid[:len(ratingsValid) // 10]


In [6]:
def Jaccard(s1, s2):
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [7]:
allbooks = set()
for user in ratingsPerUser:
    for book, _ in ratingsPerUser[user]:
        allbooks.add(book)

In [8]:
ratingsValidBinary = [(user, book, 1) for user, book, _ in ratingsValid]
for user, book, _ in ratingsValid:
    newBook = random.choice(list(allbooks))
    while newBook in ratingsPerUser[user]:
        newBook = random.choice(list(allbooks))
    ratingsValidBinary.append((user, newBook, 0))

# ratingsValidBinary = [(user, book, 1) for user, book, _ in small_ratingsValid]

# for user, book, _ in small_ratingsValid:
#     newBook = random.choice(list(allbooks))
#     while newBook in [b for b, _ in ratingsPerUser[user]]:
#         newBook = random.choice(list(allbooks))
#     ratingsValidBinary.append((user, newBook, 0))

In [9]:
ratingsTrainBinary = [(user, book, 1) for user, book, _ in ratingsTrain]
for user, book, _ in ratingsTrain:
    newBook = random.choice(list(allbooks))
    while newBook in ratingsPerUser[user]:
        newBook = random.choice(list(allbooks))
    ratingsTrainBinary.append((user, newBook, 0))



In [10]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


# Calculate max popularity once
bookCount = defaultdict(int)
for user in ratingsPerUser:
    for book, _ in ratingsPerUser[user]:
        bookCount[book] += 1
#max_popularity = max(len(ratingsPerUser[u]) for u in ratingsPerUser)

def extract_features(user, book):
    users_for_book = set(u for u, _ in ratingsPerItem[book])
    bprime = set(b for b, _ in ratingsPerUser[user] if b != book)
    
    max_jaccard = max(Jaccard(set(u for u, _ in ratingsPerItem[b]), users_for_book) for b in bprime) if bprime else 0
    popularity_score = np.log1p(bookCount[book])
    
    return [max_jaccard, popularity_score]

# def extract_features(user, book):
#     books_for_user = set(b for b, _ in ratingsPerUser[user])
#     uprime = set(u for u, _ in ratingsPerItem[book] if u != user)
    
#     max_jaccard = max(Jaccard(set(b for b, _ in ratingsPerUser[u]), books_for_user) for u in uprime) if uprime else 0
#     popularity_score = len(books_for_user) / max_popularity
    
#     return [max_jaccard, popularity_score]
# Extract features and labels for training set
features = [extract_features(user, book) for user, book, _ in ratingsTrainBinary]
labels = [predict for _, _, predict in ratingsTrainBinary]

features = np.array(features)
labels = np.array(labels)
scaler = StandardScaler()
features = scaler.fit_transform(features)
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(features, labels)


features_valid = [extract_features(user, book) for user, book, _ in ratingsValidBinary]
labels_valid = [predict for _, _, predict in ratingsValidBinary]


features_valid = np.array(features_valid)
labels_valid = np.array(labels_valid)

features_valid = scaler.transform(features_valid)

predictions = clf.predict(features_valid)
accuracy = accuracy_score(labels_valid, predictions)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.725


In [ ]:
#naive jaccard 

In [ ]:
# def solve(thresh1):
#     ans = {}
#     for user, book, _ in ratingsValidBinary:
#         users_for_book = set(u for u, _ in ratingsPerItem[book])
#         bprime = set(b for b, _ in ratingsPerUser[user] if b != book)
        
#         max_jaccard = 0
#         for b in bprime:
#             jaccard = Jaccard(set(u for u, _ in ratingsPerItem[b]), users_for_book)
#             max_jaccard = max(max_jaccard, jaccard)
        
#         ans[(user, book)] = 1 if max_jaccard > thresh1 else 0

#     correct = sum(1 for user, book, r in ratingsValidBinary if ans.get((user, book)) == r)
#     accuracy = correct / len(ratingsValidBinary) if ratingsValidBinary else 0
#     return accuracy
    


In [ ]:
# solve(0.004)

0.69855

In [80]:
#weighted jaccard

In [ ]:
# read_books = set() 
# def solve(alpha, combined_thresh):
#     read_books.clear()
#     max_popularity = max(len(ratingsPerItem[b]) for b in ratingsPerItem)

#     for user, book, _ in ratingsValidBinary:
#         users_for_book = set(u for u, _ in ratingsPerItem[book])
#         bprime = set(b for b, _ in ratingsPerUser[user] if b != book)
        
#         max_jaccard = 0
#         for b in bprime:
#             jaccard = Jaccard(set(u for u, _ in ratingsPerItem[b]), users_for_book)
#             max_jaccard = max(max_jaccard, jaccard)
             
#         popularity_score = len(users_for_book) / max_popularity
#         if(len(users_for_book) > 50):
#             read_books.add(book)
#             continue
#         combined_score = alpha * max_jaccard + (1 - alpha) * popularity_score
#         if combined_score > combined_thresh:
#             read_books.add(book) 

#     correct = 0
#     for _, book, predict in ratingsValidBinary:
#         correct += (predict == 1 and book in read_books) or (predict == 0 and book not in read_books)
    
#     acc = correct / len(ratingsValidBinary)
#     return acc


In [ ]:
#Optimal Alpha Threshold: 0.55, Optimal Combined Threshold: 0.02, Best Accuracy: 0.75725

In [157]:
print(solve(0.1, 0.032))

0.7494


In [140]:
import numpy as np
def gs(alpha_range, combined_range):
    best_accuracy = 0
    best_alpha_thresh = None
    best_combined_thresh = None
    for alpha_thresh in alpha_range:
        for combined_thresh in combined_range:
            accuracy = solve(alpha_thresh, combined_thresh)
            print(f"Alpha Threshold: {alpha_thresh}, Combined Threshold: {combined_thresh}, Accuracy: {accuracy}")
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_alpha_thresh = alpha_thresh
                best_combined_thresh = combined_thresh

    return best_alpha_thresh, best_combined_thresh, best_accuracy

alpha_range = np.arange(0.5, 0.56, 0.01) 
combined_range = np.arange(0.02, 0.024, 0.001) 

best_alpha_thresh, best_combined_thresh, best_accuracy = gs(
    alpha_range, combined_range
)
print(f"Optimal Alpha Threshold: {best_alpha_thresh}, Optimal Combined Threshold: {best_combined_thresh}, Best Accuracy: {best_accuracy}")


Alpha Threshold: 0.5, Combined Threshold: 0.02, Accuracy: 0.738
Alpha Threshold: 0.5, Combined Threshold: 0.021, Accuracy: 0.739
Alpha Threshold: 0.5, Combined Threshold: 0.022000000000000002, Accuracy: 0.7413
Alpha Threshold: 0.5, Combined Threshold: 0.023000000000000003, Accuracy: 0.742
Alpha Threshold: 0.51, Combined Threshold: 0.02, Accuracy: 0.7385
Alpha Threshold: 0.51, Combined Threshold: 0.021, Accuracy: 0.7394
Alpha Threshold: 0.51, Combined Threshold: 0.022000000000000002, Accuracy: 0.74075
Alpha Threshold: 0.51, Combined Threshold: 0.023000000000000003, Accuracy: 0.7416
Alpha Threshold: 0.52, Combined Threshold: 0.02, Accuracy: 0.73855
Alpha Threshold: 0.52, Combined Threshold: 0.021, Accuracy: 0.73925


KeyboardInterrupt: 

In [ ]:
#Optimal Alpha Threshold: 0.535, Optimal Combined Threshold: 0.022000000000000002, Best Accuracy: 0.7595

In [21]:
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    if b in ret:
        predictions.write(u + ',' + b + ",1\n")
    else:
        predictions.write(u + ',' + b + ",0\n")


predictions.close()

In [12]:
alpha = np.mean([r for _, _, r in ratingsTrain])
beta_user = defaultdict(float)
beta_book = defaultdict(float)

def solve(lambda_reg):
    for _ in range(100):
        alpha = sum(rating - (beta_user[user] + beta_book[book]) for user, book, rating in ratingsTrain) / len(ratingsTrain)
        for user, items in ratingsPerUser.items():
            beta_user[user] = sum(rating - (alpha + beta_book[book]) for book, rating in items) / (lambda_reg + len(items))
        for book, items in ratingsPerItem.items():
            beta_book[book] = sum(rating - (alpha + beta_user[user]) for user, rating in items) / (lambda_reg + len(items))

    valid_error = [(rating - (alpha + beta_user.get(user, 0) + beta_book.get(book, 0))) ** 2 for user, book, rating in ratingsValid]
    validMSE = np.mean(valid_error).item()
    return validMSE


In [ ]:
#4.342360167360211 1.4106621594632809

In [ ]:
l, r, eps = 0, 100, 1e-4

while(r-l > eps):
    m1 = l + (r-l)/3
    m2 = r - (r-l)/3
    print(m1, m2)
    if solve(m1) < solve(m2):
        r = m2
    else:
        l = m1


33.333333333333336 66.66666666666666
22.222222222222218 44.44444444444444
14.814814814814815 29.629629629629626
9.876543209876543 19.75308641975308
6.584362139917694 13.168724279835388
4.38957475994513 8.779149519890257
2.9263831732967525 5.852766346593505
4.8773052888279205 6.828227404359089
4.226997916984198 5.527612660671644
3.793459669088383 4.660536164880013
4.371510666282803 4.9495616634772235
4.17882700055133 4.564194332014276
4.050371223397014 4.307282777705645
4.221645592936102 4.3929199624751885
4.3358285059621595 4.450011418988218
4.297767534953474 4.373889476970846
4.348515496298389 4.399263457643303
4.331599509183417 4.36543148341336
4.320322184440102 4.342876833926732
4.335358617431188 4.350395050422274
4.3303464731008265 4.34037076176155
4.337029332207976 4.343712191315125


In [13]:
print(solve(4.340628291032128))

1.4101458695454359


In [14]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    prediction = alpha + beta_user.get(u, 0) + beta_book.get(b, 0)
    predictions.write(u + ',' + b + ',' + str(prediction) + '\n')
    
predictions.close()